# Compute Spectra Workflow
- Identify material system of interest
- Specify simulation parameters
- Run simulation
- Extract simulation outputs

[this cell will be deleted / cleaned up in the future]

###  Import Dependencies

In [ ]:
#data tools
import pandas as pd
import numpy as np
import qgrid
from pymatgen.ext.matproj import MPRester

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from IPython.display import HTML


### User Create ~/.mpkey.txt if it doesn't already exist

In [ ]:
Authenticate.mpkey()

## User Prompted To pick their Semiconductor of Choice
### Choice (a): query MP and fliter on properties. Dataframe is updated with each selection.

In [ ]:
# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')

display(tag)

# query MP, get all cubic systems
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

# define a function for visualizing input structures

def mpid_plot(mpid):

    # import POSCAR file

    struct = rester.get_structure_by_material_id(mpid, final = False, conventional_unit_cell=True)
    POSCAR_str = struct.to(fmt = "poscar")

    lines = POSCAR_str.split('\n')

    # get the lattice information

    lattice = lines[1]
    cell_vectors = np.array([lines[2].split() , lines[3].split() , lines[4].split()]).astype(float)

    # get the list of sites

    sites = []
    for line in lines[8:]:
        if not line:
            break
        sites.append([line.split()[0],line.split()[1],line.split()[2]])

    # convert from fractional to xyz

    sites = np.array(sites).astype(float)
    xyz = np.matmul(sites,cell_vectors).transpose()

    # get the coordinates of the box

    corners = np.array([[0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0],[0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1],[0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,1,0]]).T
    cell = np.matmul(corners,cell_vectors).transpose()
    
    box = pd.DataFrame({'x':cell[0],'y':cell[1],'z':cell[2]})

    # get a color dictionary

    elements = lines[5].split()
    atoms = lines[6].split()
    hues = ['tab:blue','tab:orange','tab:green','tab:red']
    colors = []
    for i, atom in enumerate(atoms):
        colors.extend([elements[i]]*int(atoms[i]))

    zip_iterator = zip(elements,hues)
    color_dict = dict(zip_iterator)
    
    size = 72
    POSCAR_df = pd.DataFrame({'x':xyz[0],'y':xyz[1],'z':xyz[2],'element':colors,'size':size})    
    fig = px.scatter_3d(POSCAR_df, x='x', y='y', z='z',
              color='element',labels={'x':'','y':'','z':''},size='size',size_max=size)
    
    box = px.line_3d(box,x='x', y='y', z='z')
    fig.add_traces(list(box.select_traces()))
    
    fig.update_layout(scene = dict(
                    xaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),
                    yaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),
                    zaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),),
                    width=700,
                    margin=dict(r=10, l=10, b=10, t=10)
                  )

    fig.show()
    

In [ ]:
display(tag)

# create widgets

mpid = widgets.IntText(
        value=2133,
        description='MPID:',
        disabled=False
)

plot_button = widgets.Button(
    description='plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='plot'
)

# TODO: figure out a system of taking input from EITHER mpid input, or qgrid selected data (ideally when you click on qgrid it auto-plots)
        
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        mpid_selected="mp-"+str(mpid.value)
        mpid_plot(mpid_selected)

def on_row_selected(change):
    with output:
        try:
            clear_output(wait=True)
            selected_row = mpid_widget.get_changed_df().index[mpid_widget.get_selected_rows()][0]
            mpid_selected= sc_df.at[selected_row,'task_id']
            mpid.value = mpid_selected.split('-')[1]
        except:
            mpid_selected="mp-"+str(mpid.value)
        mpid_plot(mpid_selected)


output = widgets.Output()
plot_button.on_click(on_button_clicked)
display(mpid, plot_button, output)

# if you don't, search for your structure by filtering
mpid_widget = qgrid.show_grid(sc_df)
mpid_widget.observe(on_row_selected, names=['_selected_rows'])


# filter by pretty_formula to look for a specific structure
display(mpid_widget)


### Get structure from MP database
TODO: could be combined with the last cell

In [ ]:
# This is passed to the simtool to perform simulations
# structure objects obtained directly from remote database queries must be passed to simtool as dictionaries to preserve maximum numerical precision
# quantum espresso is very sensitive to precision especially for relaxation calculations.
struct = rester.get_structure_by_material_id("mp-1002206", final = False, conventional_unit_cell=True)
struct_dict = struct.as_dict()#to(fmt="JSON")
struct_dict

### Choice (b): upload your own poscar directly. No query necessary.

In [ ]:
#users can pass their own structure as a poscar. the simtool will take the file contents as a dictionary which is produced here
struct = Structure.from_file("./POSCAR")
struct_dict = struct.as_dict()
struct_dict

# Perform structure relaxation, scf, and phonon computation and spectra extraction using simtool

### Find relax_sim simtool notebook and confirm

In [ ]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

In [ ]:
# get list of available pseudopotential files

pp_list = []

for filename in os.listdir("./simtool/pseudo/"):
    f = os.path.join("./simtool/pseudo/", filename)
    # get a list of all the PPs -- is this best instatiated here or globally?
    # if instanced here, the user could probably pass their own PPs to the constructor as well
    if os.path.isfile(f):
        pp_list.append(filename)

# filter by selected compound compositions
elements = np.unique([''.join([i for i in str(i.species) if i.isalpha()]) for i in struct.sites])
filtered_pp_list = [pp for pp in pp_list for e in elements if e in pp]

# make widgets for sim2l parameters

log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
kpoints = widgets.BoundedFloatText(
    value=6,
    min=1,
    max=20,
    step=1,
    description='kpoints:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['kpoints'].value = kpoints.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        runSim2l()
        r = Run(simToolLocation,inputs)
        results = r.getResultSummary()
        print(r.read('spectra'))
    return results
        
        
results = button.on_click(on_button_clicked)

simulation = widgets.VBox([ecutrho, ecutwfc, kpoints, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Or, iterate through inputs directly

In [ ]:
ecutwfc_list = [40]
ecutrho_list = []
kpoints_list = [3]
struct_dict_list = []
results_list = []

inputs['pps'].value = ['Si.pbe.upf', 'C.pbe.upf']
inputs['loglevel'].value = 'DEBUG'
inputs['smearing'].value = 'fixed'
inputs['numnodes'].value = 4
inputs['struct_dict'].value = struct_dict

for ecutwfc in ecutwfc_list:
    for kpoints in kpoints_list:
        inputs['ecutwfc'].value = ecutwfc
        inputs['ecutrho'].value = 4*ecutwfc
        inputs['kpoints'].value = kpoints
        print("running sim with ",ecutwfc," and ",kpoints)
        r = Run(simToolLocation,inputs)
        results_list.append(r.getResultSummary())

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)

In [ ]:
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [ ]:
results.read('spectra')

In [ ]:
print(r.read('logreport'))

In [ ]:
r.read('spectra')

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)